# Coursera Applied Data Science Final Project

#### There is no worse feeling than showing up to an early morning meeting with your stomach grumbling. Tensions are already high and being hangry isn't going to help anyone. In this problem, we are going to suppose that someone arrives via train at 30th St. Station in Philadelphia, PA. His train gets there slightly early. His meeting is at 10 am, but the train arrives around 8 am. He hasn't had anything to eat yet, but would really like to know where to go to find a good bagel shop. Reviews are incredibly important to him and he will only go somewhere if it is highly rated. He'd like to know what some good options for bagel places nearby are. This project works to solve that question by using the Foursquare API in addition to the pandas, folium, and numpy library. The goal of this project is to find the bagel places that are close to 30th St. Station, evaluate how they are rated, and plot them on a map to show our hungry user what his options are. In this project, any bagel shop that does not have any ratings recorded on the Foursquare API will not be considered. The final map will have call-out bubbles that will show the name of the bagel shop, its rating on Foursquare, and the distance (in miles) from 30th St. Station.

In [73]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [74]:
CLIENT_ID = '50JYA2FVB52KM02BAR25KMG4L5UQUKHXIBB3ASBDGANKJ4NN' # your Foursquare ID
CLIENT_SECRET = '4JYULYA0LFX2TPMJIQKE2PBWM4KR1KO2PJZMVFQWMYEB2YMT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 50JYA2FVB52KM02BAR25KMG4L5UQUKHXIBB3ASBDGANKJ4NN
CLIENT_SECRET:4JYULYA0LFX2TPMJIQKE2PBWM4KR1KO2PJZMVFQWMYEB2YMT


In [75]:
address = '3051 John F Kennedy Blvd, Philadelphia, PA 19104' #Address of 30th St. Station

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.955840430579535 -75.18398401653732


In [76]:
search_query = 'Bagels' #We'd like to find a good bagel place around 30th St. Station
radius = 10000

In [77]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=50JYA2FVB52KM02BAR25KMG4L5UQUKHXIBB3ASBDGANKJ4NN&client_secret=4JYULYA0LFX2TPMJIQKE2PBWM4KR1KO2PJZMVFQWMYEB2YMT&ll=39.955840430579535,-75.18398401653732&v=20180604&query=Bagels&radius=10000&limit=30'

In [78]:
results = requests.get(url).json() #Gets the required data from foursquare
results

{'meta': {'code': 200, 'requestId': '5e99adb50f596800268a17cf'},
 'response': {'venues': [{'id': '54ee18b7498e311c65eb024f',
    'name': 'Chestnut Street Philly Bagels',
    'location': {'address': '1705 Chestnut St',
     'lat': 39.95163783067737,
     'lng': -75.16903024578068,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.95163783067737,
       'lng': -75.16903024578068},
      {'label': 'entrance', 'lat': 39.95158, 'lng': -75.168895}],
     'distance': 1359,
     'postalCode': '19103',
     'cc': 'US',
     'city': 'Philadelphia',
     'state': 'PA',
     'country': 'United States',
     'formattedAddress': ['1705 Chestnut St',
      'Philadelphia, PA 19103',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d179941735',
      'name': 'Bagel Shop',
      'pluralName': 'Bagel Shops',
      'shortName': 'Bagels',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bagels_',
       'suffix': '.png'},
      'primary': True}],
    'deliv

In [79]:
venues = results['response']['venues'] #Selects the data of interest
df = json_normalize(venues)
df

C:\Users\blync\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.crossStreet,venuePage.id,location.neighborhood
0,54ee18b7498e311c65eb024f,Chestnut Street Philly Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,1705 Chestnut St,39.951638,-75.169030,"[{'label': 'display', 'lat': 39.95163783067737...",1359,...,"[1705 Chestnut St, Philadelphia, PA 19103, Uni...",549176,https://www.grubhub.com/restaurant/chestnut-st...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN,NaN
1,579247c9498eec1c6984de95,Fitzwater Street Philly Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,2001 Fitzwater St,39.943200,-75.175529,"[{'label': 'display', 'lat': 39.9432, 'lng': -...",1581,...,"[2001 Fitzwater St, Philadelphia, PA 19146, Un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4c4ee33f9efabe9a531fd26a,Ellen's Bagels Hoagies and More,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,200 S Broad St,39.949158,-75.164630,"[{'label': 'display', 'lat': 39.94915780832927...",1811,...,"[200 S Broad St, Philadelphia, PA 19102, Unite...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4a06e58ff964a5200a731fe3,South Street Philly Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,613 S 3rd St,39.940989,-75.147435,"[{'label': 'display', 'lat': 39.94098906897058...",3530,...,"[613 S 3rd St (btwn South & Bainbridge Sts), P...",186654,https://www.grubhub.com/restaurant/south-stree...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,btwn South & Bainbridge Sts,NaN,NaN
4,57d6c386498e3a97ac30999f,Fitzwater Street Philly Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,2001 Fitzwater St,39.943086,-75.175532,"[{'label': 'display', 'lat': 39.94308554717867...",1592,...,"[2001 Fitzwater St (at S 20th St), Philadelphi...",NaN,NaN,NaN,NaN,NaN,NaN,at S 20th St,NaN,NaN
5,5e1a12ed7497be0008fff69f,Bart’s Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,3945 Lancaster Ave,39.962684,-75.201273,"[{'label': 'display', 'lat': 39.962684, 'lng':...",1660,...,"[3945 Lancaster Ave (Brandywine Street), Phila...",NaN,NaN,NaN,NaN,NaN,NaN,Brandywine Street,NaN,NaN
6,5e1f0306d7750e0008b34e9e,Philly Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,1811 John F Kennedy Blvd,39.954258,-75.170220,"[{'label': 'display', 'lat': 39.954258, 'lng':...",1187,...,"[1811 John F Kennedy Blvd, Philadelphia, PA 19...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4ba623e0f964a520673639e3,Westmont Bagels - Deli & Cafe,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,682 W Cuthbert Blvd,39.900811,-75.063489,"[{'label': 'display', 'lat': 39.90081082764452...",11971,...,"[682 W Cuthbert Blvd (At Macarthur Blvd), Hadd...",1221678,https://www.grubhub.com/restaurant/westmont-ba...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,At Macarthur Blvd,90479469,NaN
8,4af2de93f964a520e0e821e3,Delancey Street Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,50 E Wynnewood Rd,40.001302,-75.273127,"[{'label': 'display', 'lat': 40.00130178796959...",9134,...,[50 E Wynnewood Rd (at Wynnewood Shopping Plaz...,328113,https://www.grubhub.com/restaurant/delancey-st...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,at Wynnewood Shopping Plaza,NaN,NaN
9,54635d17498e5d8de4a26795,Knead Bagels,"[{'id': '4bf58dd8d48988d179941735', 'name': 'B...",v-1587129960,False,725 Walnut at,39.948051,-75.153640,"[{'label': 'display', 'lat': 39.94805050279439...",2730,...,"[725 Walnut at, Ph

In [93]:
consoltable = df[['id', 'name', 'location.address', 'location.lat', 'location.lng', 'location.city', 'location.state']] #This selects only the data we are interested in using
consoltable

,id,name,location.address,location.lat,location.lng,location.city,location.state
0,54ee18b7498e311c65eb024f,Chestnut Street Philly Bagels,1705 Chestnut St,39.951638,-75.169030,Philadelphia,PA
1,579247c9498eec1c6984de95,Fitzwater Street Philly Bagels,2001 Fitzwater St,39.943200,-75.175529,Philadelphia,PA
2,4c4ee33f9efabe9a531fd26a,Ellen's Bagels Hoagies and More,200 S Broad St,39.949158,-75.164630,Philadelphia,PA
3,4a06e58ff964a5200a731fe3,South Street Philly Bagels,613 S 3rd St,39.940989,-75.147435,Philadelphia,PA
4,57d6c386498e3a97ac30999f,Fitzwater Street Philly Bagels,2001 Fitzwater St,39.943086,-75.175532,Philadelphia,PA
5,5e1a12ed7497be0008fff69f,Bart’s Bagels,3945 Lancaster Ave,39.962684,-75.201273,Philadelphia,PA
6,5e1f0306d7750e0008b34e9e,Philly Bagels,1811 John F Kennedy Blvd,39.954258,-75.170220,Philadelphia,PA
7,4ba623e0f964a520673639e3,Westmont Bagels - Deli & Cafe,682 W Cuthbert Blvd,39.900811,-75.063489,Haddon Township,NJ
8,4af2de93f964a520e0e821e3,Delancey Street Bagels,50 E Wynnewood Rd,40.001302,-75.273127,Wynnewood,PA
9,54635d17498e5d8de4a26795,Knead Bagels,725 Walnut at,39.948051,-75.153640,Philadelphia,PA


In [248]:
survey_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around 30th St. Station

# add 30th St. Station as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='30th St. Station',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add all bagel shops to the map as blue circle markers
for lat, lng in zip(consoltable['location.lat'], consoltable['location.lng']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(survey_map)

# display map
survey_map

In [95]:
id_numbers = consoltable['id']
id_numbers

0     54ee18b7498e311c65eb024f
1     579247c9498eec1c6984de95
2     4c4ee33f9efabe9a531fd26a
3     4a06e58ff964a5200a731fe3
4     57d6c386498e3a97ac30999f
5     5e1a12ed7497be0008fff69f
6     5e1f0306d7750e0008b34e9e
7     4ba623e0f964a520673639e3
8     4af2de93f964a520e0e821e3
9     54635d17498e5d8de4a26795
10    56b5322f498e51e59dca4e8e
11    5a63480d9e0d5424e26689fd
12    54b2a2a8498e8053a88e432a
13    4cd7ec8d7bb06dcb81719fb2
14    5798b6da498ea3541546f0e0
15    4f32a7d719836c91c7ecedda
16    4c7143b9d274b60c73dddb0d
17    4ebbee678b81c444e9535ed1
18    4ecbb1c629c26a62845708ce
19    594446905f68b90855b8ae48
20    50e4d358e4b0fa152d11bd17
21    5b002c5cc4404e001ccc3934
22    4f32b79019836c91c7f2c1dc
23    4f327e0b19836c91c7dd15b4
Name: id, dtype: object

In [83]:
#This finds the ratings for each bagel shop based on their id number
ratings = []
for id in id_numbers:
    venue_id = id
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        response = result['response']['venue']['rating']
    except:
        response = 'This venue has not been rated yet.'
    ratings = np.append(ratings, response)

In [102]:
ratings.shape

(24,)

In [114]:
#Combines the id numbers with the ratings
id_ratings = np.vstack((id_numbers, ratings)).T
id_ratingsdf = pd.DataFrame(id_ratings)
id_ratingsdf

,0,1
0,54ee18b7498e311c65eb024f,7.5
1,579247c9498eec1c6984de95,This venue has not been rated yet.
2,4c4ee33f9efabe9a531fd26a,This venue has not been rated yet.
3,4a06e58ff964a5200a731fe3,7.6
4,57d6c386498e3a97ac30999f,7.1
5,5e1a12ed7497be0008fff69f,This venue has not been rated yet.
6,5e1f0306d7750e0008b34e9e,This venue has not been rated yet.
7,4ba623e0f964a520673639e3,8.3
8,4af2de93f964a520e0e821e3,6.4
9,54635d17498e5d8de4a26795,8.8


In [118]:
#Names the columns of this table
id_ratingsdf2 = id_ratingsdf.rename(columns={0: 'id', 1: 'rating'})
id_ratingsdf2

,id,rating
0,54ee18b7498e311c65eb024f,7.5
1,579247c9498eec1c6984de95,This venue has not been rated yet.
2,4c4ee33f9efabe9a531fd26a,This venue has not been rated yet.
3,4a06e58ff964a5200a731fe3,7.6
4,57d6c386498e3a97ac30999f,7.1
5,5e1a12ed7497be0008fff69f,This venue has not been rated yet.
6,5e1f0306d7750e0008b34e9e,This venue has not been rated yet.
7,4ba623e0f964a520673639e3,8.3
8,4af2de93f964a520e0e821e3,6.4
9,54635d17498e5d8de4a26795,8.8


In [120]:
#Puts all of the data in one table
full_table = consoltable.merge(id_ratingsdf2, how='outer', on='id')
full_table

,id,name,location.address,location.lat,location.lng,location.city,location.state,rating
0,54ee18b7498e311c65eb024f,Chestnut Street Philly Bagels,1705 Chestnut St,39.951638,-75.169030,Philadelphia,PA,7.5
1,579247c9498eec1c6984de95,Fitzwater Street Philly Bagels,2001 Fitzwater St,39.943200,-75.175529,Philadelphia,PA,This venue has not been rated yet.
2,4c4ee33f9efabe9a531fd26a,Ellen's Bagels Hoagies and More,200 S Broad St,39.949158,-75.164630,Philadelphia,PA,This venue has not been rated yet.
3,4a06e58ff964a5200a731fe3,South Street Philly Bagels,613 S 3rd St,39.940989,-75.147435,Philadelphia,PA,7.6
4,57d6c386498e3a97ac30999f,Fitzwater Street Philly Bagels,2001 Fitzwater St,39.943086,-75.175532,Philadelphia,PA,7.1
5,5e1a12ed7497be0008fff69f,Bart’s Bagels,3945 Lancaster Ave,39.962684,-75.201273,Philadelphia,PA,This venue has not been rated yet.
6,5e1f0306d7750e0008b34e9e,Philly Bagels,1811 John F Kennedy Blvd,39.954258,-75.170220,Philadelphia,PA,This venue has not been rated yet.
7,4ba623e0f964a520673639e3,Westmont Bagels - Deli & Cafe,682 W Cuthbert Blvd,39.900811,-75.063489,Haddon Township,NJ,8.3
8,4af2de93f964a520e0e821e3,Delancey Street Bagels,50 E Wynnewood Rd,40.001302,-75.273127,Wynnewood,PA,6.4
9,54635d17498e5d8de4a26795,Knead Bagels,725 Walnut at,39.948051,-75.153640,Philadelphia,PA,8.8


In [122]:
full_table2 = full_table.drop(full_table[full_table['rating'] == 'This venue has not been rated yet.'].index) #Dropping all locations that don't have ratings
full_table2

,id,name,location.address,location.lat,location.lng,location.city,location.state,rating
0,54ee18b7498e311c65eb024f,Chestnut Street Philly Bagels,1705 Chestnut St,39.951638,-75.169030,Philadelphia,PA,7.5
3,4a06e58ff964a5200a731fe3,South Street Philly Bagels,613 S 3rd St,39.940989,-75.147435,Philadelphia,PA,7.6
4,57d6c386498e3a97ac30999f,Fitzwater Street Philly Bagels,2001 Fitzwater St,39.943086,-75.175532,Philadelphia,PA,7.1
7,4ba623e0f964a520673639e3,Westmont Bagels - Deli & Cafe,682 W Cuthbert Blvd,39.900811,-75.063489,Haddon Township,NJ,8.3
8,4af2de93f964a520e0e821e3,Delancey Street Bagels,50 E Wynnewood Rd,40.001302,-75.273127,Wynnewood,PA,6.4
9,54635d17498e5d8de4a26795,Knead Bagels,725 Walnut at,39.948051,-75.153640,Philadelphia,PA,8.8
10,56b5322f498e51e59dca4e8e,Einstein Bros. Bagels,5600 City Avenue,39.995774,-75.242864,Philadelphia,PA,6.8
11,5a63480d9e0d5424e26689fd,Passyunk Avenue Philly Bagels,1742 E Passyunk Ave,39.927897,-75.166014,Philadelphia,PA,8.1
12,54b2a2a8498e8053a88e432a,Philly Style Bagels,1447 E Columbia Ave,39.974429,-75.132649,Philadelphia,PA,8.2
13,4cd7ec8d7bb06dcb81719fb2,Times Square Bagels,357 W Browning Rd,39.870882,-75.099667,Bellmawr,NJ,5.9


In [124]:
#Sorts the table by the rating, highest at the top
full_table3 = full_table2.sort_values(by=['rating'], ascending=False)
full_table3

,id,name,location.address,location.lat,location.lng,location.city,location.state,rating
9,54635d17498e5d8de4a26795,Knead Bagels,725 Walnut at,39.948051,-75.153640,Philadelphia,PA,8.8
7,4ba623e0f964a520673639e3,Westmont Bagels - Deli & Cafe,682 W Cuthbert Blvd,39.900811,-75.063489,Haddon Township,NJ,8.3
12,54b2a2a8498e8053a88e432a,Philly Style Bagels,1447 E Columbia Ave,39.974429,-75.132649,Philadelphia,PA,8.2
11,5a63480d9e0d5424e26689fd,Passyunk Avenue Philly Bagels,1742 E Passyunk Ave,39.927897,-75.166014,Philadelphia,PA,8.1
18,4ecbb1c629c26a62845708ce,Redhouse Bagels,294 S White Horse Pike,39.891348,-75.066191,Audubon,NJ,7.7
3,4a06e58ff964a5200a731fe3,South Street Philly Bagels,613 S 3rd St,39.940989,-75.147435,Philadelphia,PA,7.6
0,54ee18b7498e311c65eb024f,Chestnut Street Philly Bagels,1705 Chestnut St,39.951638,-75.169030,Philadelphia,PA,7.5
19,594446905f68b90855b8ae48,Bruegger's Bagels,"8000 Essington Ave W, Terminal A",39.874517,-75.251078,Philadelphia,PA,7.2
4,57d6c386498e3a97ac30999f,Fitzwater Street Philly Bagels,2001 Fitzwater St,39.943086,-75.175532,Philadelphia,PA,7.1
10,56b5322f498e51e59dca4e8e,Einstein Bros. Bagels,5600 City Avenue,39.995774,-75.242864,Philadelphia,PA,6.8


In [125]:
#This function allows for the calculation of the distance between 2 points based on their respective latitudes and longitudes
import math

def latlondistance(lat1, lon1, lat2, lon2):
    R = 6372800  # Earth radius in meters
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [148]:
latlong_table = full_table3[['location.lat', 'location.lng']] #Creates a table of just latitudes and longitudes
latlong_table

,location.lat,location.lng
9,39.948051,-75.153640
7,39.900811,-75.063489
12,39.974429,-75.132649
11,39.927897,-75.166014
18,39.891348,-75.066191
3,39.940989,-75.147435
0,39.951638,-75.169030
19,39.874517,-75.251078
4,39.943086,-75.175532
10,39.995774,-75.242864


In [161]:
#Finds the distance between each point and 30th St. Station
lat1 = latitude
lon1 = longitude
distance = []
for lat2, lon2 in zip(latlong_table['location.lat'], latlong_table['location.lng']):
    measurement = latlondistance(lat1, lon1, lat2, lon2)
    measurement = measurement * 0.000621371
    measurement = round(measurement, 2)
    distance = np.append(distance, measurement)

distance

array([1.7 , 7.43, 3.01, 2.15, 7.67, 2.19, 0.84, 6.65, 0.99, 4.16, 5.67,
       7.38])

In [162]:
#Adds the distance column to the latitude and longitude data
latlong_table['distance'] = distance
latlong_table

C:\Users\blync\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,location.lat,location.lng,distance
9,39.948051,-75.153640,1.70
7,39.900811,-75.063489,7.43
12,39.974429,-75.132649,3.01
11,39.927897,-75.166014,2.15
18,39.891348,-75.066191,7.67
3,39.940989,-75.147435,2.19
0,39.951638,-75.169030,0.84
19,39.874517,-75.251078,6.65
4,39.943086,-75.175532,0.99
10,39.995774,-75.242864,4.16


In [165]:
full_table_dist = full_table3.merge(latlong_table, how='outer', on='location.lat')
full_table_dist2 = full_table_dist.drop(columns=['location.lng_y'])

In [180]:
full_table_dist3 = full_table_dist2.rename(columns={'id': 'ID', 'name': 'Name', 'location.address': 'Address', 'location.lat': 'Latitude', 'location.lng_x':'Longitude', 'location.city':'City', 'location.state':'State','rating':'Rating','distance':'Distance'})
full_table_dist3
#Merges all data into one dataset

,ID,Name,Address,Latitude,Longitude,City,State,Rating,Distance
0,54635d17498e5d8de4a26795,Knead Bagels,725 Walnut at,39.948051,-75.153640,Philadelphia,PA,8.8,1.70
1,4ba623e0f964a520673639e3,Westmont Bagels - Deli & Cafe,682 W Cuthbert Blvd,39.900811,-75.063489,Haddon Township,NJ,8.3,7.43
2,54b2a2a8498e8053a88e432a,Philly Style Bagels,1447 E Columbia Ave,39.974429,-75.132649,Philadelphia,PA,8.2,3.01
3,5a63480d9e0d5424e26689fd,Passyunk Avenue Philly Bagels,1742 E Passyunk Ave,39.927897,-75.166014,Philadelphia,PA,8.1,2.15
4,4ecbb1c629c26a62845708ce,Redhouse Bagels,294 S White Horse Pike,39.891348,-75.066191,Audubon,NJ,7.7,7.67
5,4a06e58ff964a5200a731fe3,South Street Philly Bagels,613 S 3rd St,39.940989,-75.147435,Philadelphia,PA,7.6,2.19
6,54ee18b7498e311c65eb024f,Chestnut Street Philly Bagels,1705 Chestnut St,39.951638,-75.169030,Philadelphia,PA,7.5,0.84
7,594446905f68b90855b8ae48,Bruegger's Bagels,"8000 Essington Ave W, Terminal A",39.874517,-75.251078,Philadelphia,PA,7.2,6.65
8,57d6c386498e3a97ac30999f,Fitzwater Street Philly Bagels,2001 Fitzwater St,39.943086,-75.175532,Philadelphia,PA,7.1,0.99
9,56b5322f498e51e59dca4e8e,Einstein Bros. Bagels,5600 City Avenue,39.995774,-75.242864,Philadelphia,PA,6.8,4.16


In [242]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around 30th St. Station

# add 30th St. Station as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='30th St. Station',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, name, rating, distance in zip(full_table_dist3.Latitude, full_table_dist3.Longitude, full_table_dist3.Name, full_table_dist3.Rating, full_table_dist3.Distance):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=('Name: ' + name[:8] + '<br>' + name[9:25]+ '<br>' + name[26:] + '<br>'  #For some reason, I could not include characters 8 and 25.  It did not work in my code             
            'Rating: ' + str(rating) + '<br>' 
            'Distance: ' + str(distance) + ' mi'),
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map